In [1]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import numpy as np
import pandas as pd
from gensim.downloader import load
from numbers_parser import Document
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
import pandas as pd

# Load the file to see its contents
file_path = 'stream urls - Blad1.numbers'

In [3]:
# Load the .numbers file
file_path = 'stream urls - Blad1.numbers'
doc = Document(file_path)

# Get the sheets
sheets = doc.sheets

# Extract the first sheet
first_sheet = sheets[0]

# Extract all data from the first table in the sheet
data = []
for table in first_sheet.tables:
    for row in table.rows():
        data.append([cell.value for cell in row])

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# Define the columns that contain the words
words_col_1 = 'Madikwe Game Reserve, South Africa'
words_col_2 = 'elephant, giraffe, buffalo, zebra, hippo, rhino, lion, leopard, hyena, wildebeest'
words_col_3 = 'africa'

# Create a new DataFrame to hold the transformed data
transformed_data = []

# Process each row in the DataFrame
for index, row in df.iterrows():
    # Check for None values in the columns of interest
    if (row[words_col_1] is None) or (row[words_col_2] is None) or (row[words_col_3] is None):
        continue
    
    words2 = [word.strip() for word in row[words_col_2].split(',') if word.strip()]
    
    for word2 in words2:
        transformed_data.append([word2, row[words_col_3]])

# Convert the transformed data to a new DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=[words_col_2, words_col_3])

# Display the transformed DataFrame
transformed_df.head()


,"elephant, giraffe, buffalo, zebra, hippo, rhino, lion, leopard, hyena, wildebeest",africa
0,red panda,other
1,shark,sea
2,sea lion,sea
3,eagle,other
4,owl,other


In [4]:
# Rename columns
transformed_df = transformed_df.rename(columns={
    'elephant, giraffe, buffalo, zebra, hippo, rhino, lion, leopard, hyena, wildebeest': 'animals',
    'africa': 'region'
})

In [5]:
transformed_df.head(20)

,animals,region
0,red panda,other
1,shark,sea
2,sea lion,sea
3,eagle,other
4,owl,other
5,eagle,other
6,penguin,other
7,Elephant,africa
8,Rhino,africa
9,lion,africa


In [6]:
transformed_df["animals"].unique()

array(['red panda', 'shark', 'sea lion', 'eagle', 'owl', 'penguin',
       'Elephant', 'Rhino', 'lion', 'leopards', 'buffalo', 'wildebeest',
       'zebra', 'giraffe', 'hippo', 'flamingo', 'Gorilla', 'elephants',
       'hippos', 'giraffes', 'zebras', 'gazelles', 'crocodiles',
       'sea cow', 'fish', 'ray', 'moray eel', 'turtle', 'bison', 'oryx',
       'rhino', 'elephant', 'deer', 'wild pig', 'jellyfish', 'orca',
       'birds', 'squirrel', 'deer (bushbucks', 'elands)',
       'wild pigs (incl hogs)', 'and giraffes', 'various reptiles',
       'leopard', 'hyena', 'kudu', 'monkey', 'warthog', 'wild dog',
       'Buffalo', 'Bushbuck', 'Bushpig', 'Common Reedbuck', 'Giraffe',
       'Grey Duiker', 'Hippo', 'Impala', 'Kudu', 'Leopard', 'Lion',
       'Nyala', 'Porcupine', 'Red Duiker', 'Side stripped jackal',
       'Steenbuck', 'Suni', 'Warthog', 'Waterbuck', 'Wild dog',
       'White rhino', 'Wildebeest', 'Zebra', 'predators',
       'hippo and buffalo', 'buffalo and kudu', 'seal', 'l

In [7]:
# Example DataFrame
data = {
    'animals': ['red panda', 'shark', 'sea lion', 'eagle', 'owl', 'eagle', 'penguin', 'Elephant', 'Rhino', 'lion', 
                'leopards', 'buffalo', 'wildebeest', 'zebra', 'giraffe', 'hippo', 'flamingo', 'Elephant', 'Rhino', 'lion'],
    'region': ['other', 'sea', 'sea', 'other', 'other', 'other', 'other', 'africa', 'africa', 'africa', 'africa', 
               'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa']
}
df = pd.DataFrame(data)

# Prepare the sentences for Word2Vec
sentences = df.groupby('region')['animals'].apply(list).tolist()


In [8]:
sentences

[['Elephant',
  'Rhino',
  'lion',
  'leopards',
  'buffalo',
  'wildebeest',
  'zebra',
  'giraffe',
  'hippo',
  'flamingo',
  'Elephant',
  'Rhino',
  'lion'],
 ['red panda', 'eagle', 'owl', 'eagle', 'penguin'],
 ['shark', 'sea lion']]

In [13]:
# # Example DataFrame
# data = {
#     'animals': ['red panda', 'shark', 'sea lion', 'eagle', 'owl', 'eagle', 'penguin', 'Elephant', 'Rhino', 'lion', 
#                 'leopards', 'buffalo', 'wildebeest', 'zebra', 'giraffe', 'hippo', 'flamingo', 'Elephant', 'Rhino', 'lion'],
#     'region': ['other', 'sea', 'sea', 'other', 'other', 'other', 'other', 'africa', 'africa', 'africa', 'africa', 
#                'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa', 'africa']
# }
# df = pd.DataFrame(data)

# # Add region as context word to each animal and repeat animals to emphasize regional context
# region_sentences = []
# for region, group in df.groupby('region'):
#     region_sentences.append([region] + group['animals'].tolist() * 20)

# # Train Word2Vec model with region context
# word2vec_model = Word2Vec(region_sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Get word embeddings
# def get_word_embedding(word):
#     return word2vec_model.wv[word]

# # Example words
# words = df['animals'].unique()

# # Get embeddings
# embeddings = {word: get_word_embedding(word) for word in words}

# # Calculate similarities
# similarities = {}
# for word1 in words:
#     for word2 in words:
#         if word1 != word2:
#             similarities[(word1, word2)] = cosine_similarity([embeddings[word1]], [embeddings[word2]])[0][0]

# # Print similarities
# for pair, similarity in similarities.items():
#     print(f"Similarity between {pair[0]} and {pair[1]}: {similarity:.4f}")


Similarity between red panda and shark: -0.1126
Similarity between red panda and sea lion: 0.2854
Similarity between red panda and eagle: 0.1349
Similarity between red panda and owl: 0.1031
Similarity between red panda and penguin: -0.0422
Similarity between red panda and Elephant: 0.0220
Similarity between red panda and Rhino: 0.1950
Similarity between red panda and lion: -0.0435
Similarity between red panda and leopards: 0.0847
Similarity between red panda and buffalo: 0.0379
Similarity between red panda and wildebeest: -0.0457
Similarity between red panda and zebra: -0.0673
Similarity between red panda and giraffe: 0.1183
Similarity between red panda and hippo: 0.0572
Similarity between red panda and flamingo: -0.0320
Similarity between shark and red panda: -0.1126
Similarity between shark and sea lion: 0.0553
Similarity between shark and eagle: 0.1120
Similarity between shark and owl: -0.0147
Similarity between shark and penguin: 0.0777
Similarity between shark and Elephant: 0.1304

In [45]:
# Load pre-trained Word2Vec model
model_path = 'GoogleNews-vectors-negative300.bin' 
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

# Example animals
animals = ['redPanda', 'shark', 'seaLion', 'eagle', 'owl', 'penguin',
       'Elephant', 'Rhino', 'lion', 'leopard', 'buffalo', 'wildebeest',
       'giraffe', 'hippo', 'flamingo', 'Gorilla', 'elephant',
       'zebra', 'gazelle', 'crocodile',
       'sea cow', 'fish', 'ray', 'morayEel', 'turtle', 'bison', 'oryx',
       'rhino', 'elephant', 'deer', 'wildpig', 'jellyfish', 'orca',
       'birds', 'squirrel',
       'wildpig', 'reptile',
       'leopard', 'hyena', 'kudu', 'monkey', 'warthog', 'wilddog',
       'Buffalo', 'Bushbuck', 'Bushpig', 'Reedbuck', 'Giraffe',
       'GreyDuiker', 'Hippo', 'Impala', 'Kudu', 'Leopard', 'Lion',
       'Nyala', 'Porcupine', 'RedDuiker',
       'Steenbuck', 'Suni', 'Warthog', 'Waterbuck', 'Wilddog',
       'Whiterhino', 'Wildebeest', 'Zebra', 'predator',
       'hippo', 'buffalo', 'seal', 'lion', 'rhino',
       'Giraffe', 'antelope', 'genet', 'squirrel',
       'warthog', 'Giantkingfisher', 'wolf', 'cheetah', 'impala',
       'baboons', 'owl', 'Egyptiangeese',
       'ray', 'turtle', 'hyenas', 'kudus', 'Quail', 'Javelina',
       'MuleDeer', 'Coyote', 'ProngHorn']

# Get word embeddings
embeddings = {animal: word2vec_model[animal] for animal in animals if animal in word2vec_model}

def get_top_similar_words(input_word, top_n=5):
    if input_word not in word2vec_model:
        return f"The word '{input_word}' is not in the model."

    input_embedding = word2vec_model[input_word]
    similarities = {}

    for animal, embedding in embeddings.items():
        if animal != input_word:
            similarity = cosine_similarity([input_embedding], [embedding])[0][0]
            similarities[animal] = similarity

    # Sort similarities and get the top N
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:top_n]

    return sorted_similarities

# Example usage
input_word = 'lion' 
top_similar_words = get_top_similar_words(input_word)

print(f"Top similar words to '{input_word}':")
for word, similarity in top_similar_words:
    print(f"{word}: {similarity:.4f}")


Top similar words to 'lion':
elephant: 0.5120
hyena: 0.4852
leopard: 0.4527
wolf: 0.4486
monkey: 0.4475


In [30]:
#takes more time to run


def load_glove_model(glove_file):
    model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            try:
                embedding = np.array([float(val) for val in split_line[1:]])
                model[word] = embedding
            except ValueError:
                # Skip the line if it contains non-numeric values
                continue
    return model

glove_model = load_glove_model('glove.840B.300d.txt')  # Path to the pre-trained model

# Example animals
animals = ['redPanda', 'shark', 'seaLion', 'eagle', 'owl', 'penguin',
       'Elephant', 'Rhino', 'lion', 'leopard', 'buffalo', 'wildebeest',
       'giraffe', 'hippo', 'flamingo', 'Gorilla', 'elephant',
       'zebra', 'gazelle', 'crocodile',
       'sea cow', 'fish', 'ray', 'morayEel', 'turtle', 'bison', 'oryx',
       'rhino', 'elephant', 'deer', 'wildpig', 'jellyfish', 'orca',
       'birds', 'squirrel',
       'wildpig', 'reptile',
       'leopard', 'hyena', 'kudu', 'monkey', 'warthog', 'wilddog',
       'Buffalo', 'Bushbuck', 'Bushpig', 'Reedbuck', 'Giraffe',
       'GreyDuiker', 'Hippo', 'Impala', 'Kudu', 'Leopard', 'Lion',
       'Nyala', 'Porcupine', 'RedDuiker',
       'Steenbuck', 'Suni', 'Warthog', 'Waterbuck', 'Wilddog',
       'Whiterhino', 'Wildebeest', 'Zebra', 'predator',
       'hippo', 'buffalo', 'seal', 'lion', 'rhino',
       'Giraffe', 'antelope', 'genet', 'squirrel',
       'warthog', 'Giantkingfisher', 'wolf', 'cheetah', 'impala',
       'baboons', 'owl', 'Egyptiangeese',
       'ray', 'turtle', 'hyenas', 'kudus', 'Quail', 'Javelina',
       'MuleDeer', 'Coyote', 'ProngHorn']

# Get word embeddings
embeddings = {animal: glove_model[animal] for animal in animals if animal in glove_model}

def get_top_similar_words(input_word, top_n=5):
    if input_word not in glove_model:
        return f"The word '{input_word}' is not in the model."

    input_embedding = glove_model[input_word]
    similarities = {}

    for animal, embedding in embeddings.items():
        similarity = cosine_similarity([input_embedding], [embedding])[0][0]
        similarities[animal] = similarity

    # Sort similarities and get the top N
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:top_n]

    return sorted_similarities

# Example usage
input_word = 'africa'  # Replace with the word you want to input
top_similar_words = get_top_similar_words(input_word)

print(f"Top similar words to '{input_word}':")
for word, similarity in top_similar_words:
    print(f"{word}: {similarity:.4f}")


Top similar words to 'africa':
elephant: 0.4039
lion: 0.3928
zoo: 0.3207
bird: 0.3161
eagle: 0.3154
